In [ ]:
#pip install numpy scipy matplotlib scikit-image opencv-python tqdm SimpleITK tifffile

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import os
import time
from klt_log_demons import KLTLogDemonsCorrection, motion_correct_video
from scipy import ndimage

def main():
    """
    Example of how to use the KLT-Log-Demons motion correction module with your own data.
    """
    # Path to your data file - update this with your file path
    input_file_path = r"F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\CAAR part2 data\disinhibition\New folder (2)\Disinhib1_7.23.20_contra3_25um_cor.tif"
    
    # Set output directory
    output_dir = r"F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\Logdemonsoutput\LDoutput_1"
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Loading data from: {input_file_path}")
    # Load your data
    try:
        # For TIFF files
        video_data = imread(input_file_path)
        
        # If your data has a different format, uncomment and modify one of these:
        # For numpy arrays
        # video_data = np.load(input_file_path)
        
        # For other formats, you might need additional libraries
        # import h5py
        # with h5py.File(input_file_path, 'r') as f:
        #     video_data = f['dataset_name'][:]
    except Exception as e:
        print(f"Error loading data: {e}")
        return
    
    # Check if data loaded properly
    if video_data is None or video_data.size == 0:
        print("Failed to load data or data is empty")
        return
    
    print(f"Data loaded successfully: shape={video_data.shape}, dtype={video_data.dtype}")
    
    # Ensure data is 3D [frames, height, width]
    if len(video_data.shape) > 3:
        print(f"Data has {len(video_data.shape)} dimensions. Using first channel/volume.")
        # For 4D data [time, z, y, x] or [time, channel, y, x]
        if video_data.shape[1] <= 10:  # Likely channels or z-slices
            # Taking first channel/z-slice
            video_data = video_data[:, 0, :, :]
        else:
            print("Cannot determine how to reshape data. Please reshape data to [frames, height, width].")
            return
    
    # Convert to float32 if needed
    if video_data.dtype != np.float32:
        # Normalize to 0-1 range if data is integer type
        if np.issubdtype(video_data.dtype, np.integer):
            video_data = video_data.astype(np.float32) / np.iinfo(video_data.dtype).max
        else:
            video_data = video_data.astype(np.float32)
        print(f"Converted data to float32 with range [{video_data.min()}, {video_data.max()}]")
    
    # Set parameters - adjust these based on your data characteristics
    template_idx = 0  # Use first frame as template, change if needed
    max_iters = 50    # Maximum iterations for log-demons
    sigma_fluid = 2.0 # Fluid regularization parameter
    sigma_diffusion = 2.0  # Diffusion regularization parameter
    use_klt = True    # Use KLT for initial rigid correction
    n_processes = 4   # Number of parallel processes
    
    print(f"Starting motion correction with {n_processes} processes...")
    start_time = time.time()
    
    # Run motion correction
    corrected_frames, velocity_fields, rigid_transforms = motion_correct_video(
        video_data, 
        template_idx=template_idx, 
        max_iters=max_iters,
        sigma_fluid=sigma_fluid,
        sigma_diffusion=sigma_diffusion,
        use_klt=use_klt,
        n_processes=n_processes,
        output_dir=output_dir,
        save_results=True
    )
    
    elapsed_time = time.time() - start_time
    print(f"Motion correction completed in {elapsed_time:.2f} seconds")
    
    # Save corrected video as TIFF
    print("Saving corrected video as TIFF...")
    corrected_file_path = os.path.join(output_dir, 'corrected_video.tif')
    imwrite(corrected_file_path, 
            corrected_frames.astype(np.float32),
            photometric='minisblack')
    
    print(f"Corrected video saved to: {corrected_file_path}")
    print(f"All results saved to: {output_dir}")
    print("Done!")

def visualize_motion(rigid_transforms, output_dir):
    """
    Visualize the estimated rigid motion trajectories.
    
    Parameters
    ----------
    rigid_transforms : numpy.ndarray
        Array of rigid transformations with shape (n_frames, 2)
    output_dir : str
        Directory to save visualization
    """
    plt.figure(figsize=(10, 6))
    
    # Plot x and y displacements over time
    frames = np.arange(len(rigid_transforms))
    
    plt.subplot(2, 1, 1)
    plt.plot(frames, rigid_transforms[:, 0], 'r-', label='X displacement')
    plt.title('X Displacement Over Time')
    plt.xlabel('Frame')
    plt.ylabel('Displacement (pixels)')
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(frames, rigid_transforms[:, 1], 'b-', label='Y displacement')
    plt.title('Y Displacement Over Time')
    plt.xlabel('Frame')
    plt.ylabel('Displacement (pixels)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'rigid_motion_trajectory.png'))
    plt.close()
    
    # Plot trajectory in 2D
    plt.figure(figsize=(8, 8))
    plt.plot(rigid_transforms[:, 0], rigid_transforms[:, 1], 'k-')
    plt.plot(rigid_transforms[:, 0], rigid_transforms[:, 1], 'r.', markersize=5)
    plt.plot(rigid_transforms[0, 0], rigid_transforms[0, 1], 'go', markersize=10, label='Start')
    plt.plot(rigid_transforms[-1, 0], rigid_transforms[-1, 1], 'bo', markersize=10, label='End')
    
    plt.title('Motion Trajectory')
    plt.xlabel('X Displacement (pixels)')
    plt.ylabel('Y Displacement (pixels)')
    plt.grid(True)
    plt.legend()
    plt.axis('equal')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'motion_trajectory_2d.png'))
    plt.close()

if __name__ == "__main__":
    main()

Loading data from: F:\Recovered\Research\BoninLab\PainModelingProject\calcium_imaging_data\CAAR Testing\CAAR part2 data\disinhibition\New folder (2)\Disinhib1_7.23.20_contra3_25um_cor.tif
Data loaded successfully: shape=(589, 1080, 1920), dtype=uint16
Converted data to float32 with range [0.0, 0.01879911497235298]
Starting motion correction with 4 processes...


  0%|                                                                                          | 0/1 [1:30:25<?, ?it/s]


MemoryError: Unable to allocate 7.91 MiB for an array with shape (1080, 1920) and data type float32